In [9]:
import pandas as pd
import psycopg2
import pandas.io.sql as sqlio
import plotly.express as px
import dash
import dash_core_components as dcc
import dash_html_components as html
psw=''

In [10]:


def porcentaje_genero():
    # Establecer la conexión a la base de datos
    engine = psycopg2.connect(
        dbname="p2",
        user="postgres",
        password=psw,
        host="proyecto2.c9pexl84mjtw.us-east-1.rds.amazonaws.com",
        port="5432"
    )

    # Definir la consulta SQL
    query = """
    SELECT g AS genero, 
           SUM(CASE WHEN target = 1 THEN 1 ELSE 0 END) AS graduados
    FROM grado
    WHERE g IN (0, 1) -- Filtrar por géneros 0 (Mujer) y 1 (Hombre)
    GROUP BY g;
    """

    # Ejecutar la consulta y obtener el resultado como un DataFrame
    df = sqlio.read_sql_query(query, engine)

    # Reemplazar 0 con "Mujeres" y 1 con "Hombres" en la columna "genero"
    df['genero'] = df['genero'].map({0: 'Mujeres', 1: 'Hombres'})

    # Calcular el porcentaje de graduados y desertores
    df['Porcentaje de Graduados'] = (df['graduados'] / df['graduados'].sum()) * 100
    df['Porcentaje de Desertores'] = 100 - df['Porcentaje de Graduados']

    # Crear la figura de la gráfica de barras
    fig = px.bar(
        df, x='genero', y=['Porcentaje de Graduados', 'Porcentaje de Desertores'],
        labels={'value': 'Porcentaje', 'genero': 'Género'},
        title='Porcentaje de Graduados y Desertores por Género',
        color_discrete_map={'Porcentaje de Graduados': 'green', 'Porcentaje de Desertores': 'lightgreen'},
    )

    # Crear la aplicación Dash
    app = dash.Dash(__name__)

    app.layout = html.Div([
        dcc.Graph(figure=fig)
    ])

    # Ejecutar la aplicación Dash
    if __name__ == '__main__':
        app.run_server(debug=True)

# Llamar a la función para ejecutar el código
porcentaje_genero()


C:\Users\Sebastian\AppData\Local\Temp\ipykernel_16808\224195722.py:21: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [7]:
def porcentaje_graduados_por_curso():
    #CONECTARSE
    import psycopg2
    import pandas as pd
    import sqlite3
    engine = psycopg2.connect(
        dbname="p2",
        user="postgres",
        password=psw,
        host="proyecto2.c9pexl84mjtw.us-east-1.rds.amazonaws.com",
        port="5432"
    )
    cursor = engine.cursor()

    query = """
    SELECT C AS curso,
        SUM(CASE WHEN target = 1 THEN 1 ELSE 0 END) AS graduados
    FROM grado
    WHERE C BETWEEN 1 AND 17  -- Filtrar por cursos del 1 al 17
    GROUP BY C
    ORDER BY C;
    """

    # Mapeo de números de curso a nombres
    curso_nombre_map = {
        1: "Biofuel Production Technologies",
        2: "Animation and Multimedia Design",
        3: "Social Service (evening attendance)",
        4: "Agronomy",
        5: "Communication Design",
        6: "Veterinary Nursing",
        7: "Informatics Engineering",
        8: "Equinculture",
        9: "Management",
        10: "Social Service",
        11: "Tourism",
        12: "Nursing",
        13: "Oral Hygiene",
        14: "Advertising and Marketing Management",
        15: "Journalism and Communication",
        16: "Basic Education",
        17: "Management (evening attendance)"
    }

    # Ejecutar la consulta SQL y cargar los resultados en un DataFrame
    df = pd.read_sql_query(query, engine)

    # Mapear los números de curso a nombres
    df['curso'] = df['curso'].map(curso_nombre_map)


    # Calcular el porcentaje de graduados y no graduados
    df['Graduados'] = (df['graduados'] / df['graduados'].sum()) * 100
    df['Desertores'] = 100 - df['Graduados']

    # Crear la figura de la gráfica de barras horizontales
    fig = px.bar(
        df, y='curso', x=['Graduados', 'Desertores'],
        labels={'value': 'Porcentaje', 'curso': 'Curso'}, title='Porcentaje de Graduados y Desertores por Curso',
        color_discrete_map={'Graduados': 'green', 'Desertores': 'lightgreen'},
        orientation='h'
    )

    # Crear la aplicación Dash
    app = dash.Dash(__name__)

    app.layout = html.Div([
        dcc.Graph(figure=fig)
    ])

    if __name__ == '__main__':
        app.run_server(debug=True)

# Ejecutar la función
porcentaje_graduados_por_curso()

C:\Users\Sebastian\AppData\Local\Temp\ipykernel_16808\2033440614.py:46: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [8]:

def porc_edades():
    #CONECTARSE
    import psycopg2
    import pandas as pd
    import sqlite3
    engine = psycopg2.connect(
        dbname="p2",
        user="postgres",
        password=psw,
        host="proyecto2.c9pexl84mjtw.us-east-1.rds.amazonaws.com",
        port="5432"
    )
    cursor = engine.cursor()
    query = """
    SELECT AE AS age,
       SUM(CASE WHEN target = 1 THEN 1 ELSE 0 END) AS graduados,
       SUM(CASE WHEN target = 0 THEN 1 ELSE 0 END) AS no_graduados
    FROM grado
    GROUP BY AE;
    """
    df = pd.read_sql_query(query, engine)

    # Calcular los porcentajes
    df['Graduados'] = (df['graduados'] / (df['graduados'] + df['no_graduados'])) * 100
    df['Desertores'] = (df['no_graduados'] / (df['graduados'] + df['no_graduados'])) * 100

    # Mapear los valores de age a etiquetas más descriptivas
    df['age'] = df['age'].map({1: 'Jóvenes', 2: 'Adultos', 3: 'Mayores'})

    # Crear una nueva columna con el rango de edad
    df['Rango de Edad'] = df['age'].map({'Jóvenes': '16-30 años', 'Adultos': '31-45 años', 'Mayores': '46-63 años'})

    # Crear la figura de la gráfica de barras con colores verdes y leyenda personalizada
    fig = px.bar(
        df, x='Rango de Edad', y=['Graduados', 'Desertores'],
        labels={'value': 'Porcentaje'}, title='Porcentaje de Graduados y Desertores por Rango de Edad',
        color_discrete_map={'Graduados': 'green', 'Desertores': 'lightgreen'},
    )

    # Crear la aplicación Dash
    app = dash.Dash(__name__)

    app.layout = html.Div([
        dcc.Graph(figure=fig)
    ])

    if __name__ == '__main__':
        app.run_server(debug=True)

# Ejecutar la función
porc_edades()




C:\Users\Sebastian\AppData\Local\Temp\ipykernel_16808\2509575732.py:21: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

